# Task 1

## Data Model

As Station-Date is a 1-many relationship;
    Date-Climate is a 1-1 relationship and
    Date-Hotspot is a 1-0/many relationship.
The data models are designed as the following:

```
Station Collection:
    Stations = [
        {
            id: station,
            dates: [id of date]
        }
    ]
Dates Collection:
    Dates = [
        {
            id: date,
            climate: {
                // climate attributes
            },
            hotspots: [id of hotspot]
        }
    ]
Hotspots Collection:
    Hotspots = [
        {
            id: id,
            other_attributes: ...
        }
    ]
```

## Justification
### Benefits
- Decoupling Station from Date allows Dates to be referenced from Station
- Putting the data of climate in Dates allows direct access to climate from a given data
- Decoupling Dates from Hotspots because transitive dependency thing (datetime -> date)

### Drawback
- Only Year 2022 contains data of hotspots, so there are a lot of empty list for date.hotspots


# 2. Queueing MongoDB with PyMongo
## 2.1 Read data and create database

### Importing Libraries

In [108]:
import pandas as pd             # pandas version 2.0.1
from pymongo import MongoClient
from datetime import datetime
from bson import ObjectId
from pprint import pprint

### Connecting to Database

In [109]:
client = MongoClient('mongodb://localhost:27017/')
db = client.fit3182_assignment_db
db.stations.drop()
db.dates.drop()
db.hotspots.drop()

### Reading Datasets

In [110]:
climate_historic = pd.read_csv("../dataset/climate_historic.csv")
hotspot_historic = pd.read_csv("../dataset/hotspot_historic.csv")

### Functions for converting date in string to datetime

In [111]:
def raw_date_to_datetime(date: str) -> datetime:
    dd, mm, yyyy = date.split("/")
    return datetime(int(yyyy), int(mm), int(dd))

def raw_datetime_to_datatime(datetime_str: str) -> datetime:
    date, time = datetime_str.split("T")
    yy, mm, dd = date.split("-")
    h, m, s = time.split(":")
    return datetime(int(yy), int(mm), int(dd), hour=int(h), minute=int(m), second=int(s))

### Light data wrangling

In [112]:
dates = pd.DataFrame(climate_historic.date).merge(hotspot_historic.date).date           # merging both datasets' date
dates = pd.DataFrame(dates.unique())                                                    # Get all unique dates from both data set
dates = list(dates[0].apply(raw_date_to_datetime))                                      # date:str -> date:datetime; Series -> List

climate_historic.date = climate_historic.date.apply(raw_date_to_datetime)               # date:str -> date:datetime
hotspot_historic.date = hotspot_historic.date.apply(raw_date_to_datetime)               # date:str -> date:datetime
hotspot_historic.datetime = hotspot_historic.datetime.apply(raw_datetime_to_datatime)   # datetime:str -> datetime:datetime

### Creating Data Models

In [113]:
# list representing the Station collection
stations_col = [
    {
        "_id": int(station),
        "dates": [
            {"date": date}
            for date in climate_historic[climate_historic.station == station]["date"].array
        ]
    }
    for station in list(climate_historic.station.unique())
]

In [114]:
# list representing the Hotspot collection
hotspots_col = [
    {
        "_id": ObjectId(),              # this is done as there are duplicated value for "date" and "datetime" across rows of data
        "date": hotspot.date,           # keeping date is intentional, as this works better for data wrangling
        "datetime": hotspot.datetime,   # mongoDB doesnt have a Time object, this both date and datetime are kept.
        "lat": hotspot.latitude,
        "lng": hotspot.longitude,
        "confidence": hotspot.confidence,
        "surface_temperature": hotspot.surface_temperature_celcius
    }
    for _, hotspot in hotspot_historic.iterrows()
]

In [115]:
# list representing the Dates collection
dates_col = [
    {
        "_id": date,
        "climate": {    # date-climate is 1-1
            "air_temperature": int(climate_historic[climate_historic.date == date]["air_temperature_celcius"].array[0]),
            "relative_humidity": float(climate_historic[climate_historic.date == date]["relative_humidity"].array[0]),
            "windspeed_knots": float(climate_historic[climate_historic.date == date]["windspeed_knots"].array[0]),
            "max_wind_speed": float(climate_historic[climate_historic.date == date]["max_wind_speed"].array[0]),
            "precipitation": climate_historic[climate_historic.date == date]["precipitation"].array[0],                     # str
            "ghi": int(climate_historic[climate_historic.date == date]["GHI_w/m2"].array[0])
        },
        "hotspots": [   # date-hotspots is 1-0/many
            {"hotspot_id": hotspot}
            for hotspot in pd.DataFrame(hotspots_col)[pd.DataFrame(hotspots_col).date == date]["_id"]
        ]
    }
    for date in dates
]

### Inserting Data to MongoDb

In [116]:
from pymongo.errors import BulkWriteError

try:
    db.stations.insert_many(stations_col)
except BulkWriteError:
    print("Duplicated Keys in stations (Data already inserted)")
else:
    print("Station inserted")

try:
    db.dates.insert_many(dates_col)
except BulkWriteError:
    print("Duplicated Keys in dates (Data already inserted)")
else:
    print("Date inserted")

try:
    db.hotspots.insert_many(hotspots_col)
except BulkWriteError:
    print("Duplicated Keys in hotspots (Data already inserted)")
else:
    print("Hotspot inserted")

#db.hotspots.update_many({}, {"$unset": {"date": 1}})    # dropping the date column as it is only used for the cell above

Station inserted
Date inserted
Hotspot inserted


## 2.2 Querying the database
### 2.2a Finding the climate data on 12th December 2022

In [117]:
pprint(
    db.dates.find_one(
        {"_id": datetime(2022, 12, 12)},
        {"_id": 1, "climate": 1}
    )
)

{'_id': datetime.datetime(2022, 12, 12, 0, 0),
 'climate': {'air_temperature': 19,
             'ghi': 156,
             'max_wind_speed': 12.0,
             'precipitation': ' 0.00I',
             'relative_humidity': 55.3,
             'windspeed_knots': 6.2}}


### 2.2b Finding hotspot data when surface temperature between 65 and 100

In [118]:
for hotspot in db.hotspots.find(
    {"surface_temperature": {"$gt": 65, "$lt": 100}},
    {"surface_temperature":1, "lat":1, "lng":1, "confidence":1, "_id":0}
):
    pprint(hotspot)

{'confidence': 78, 'lat': -37.966, 'lng': 145.051, 'surface_temperature': 68}
{'confidence': 86, 'lat': -35.543, 'lng': 143.316, 'surface_temperature': 67}
{'confidence': 93, 'lat': -37.875, 'lng': 142.51, 'surface_temperature': 73}
{'confidence': 95, 'lat': -37.613, 'lng': 149.305, 'surface_temperature': 75}
{'confidence': 90, 'lat': -37.624, 'lng': 149.314, 'surface_temperature': 66}
{'confidence': 93, 'lat': -38.057, 'lng': 144.211, 'surface_temperature': 73}
{'confidence': 92, 'lat': -37.95, 'lng': 142.366, 'surface_temperature': 70}
{'confidence': 100, 'lat': -36.282, 'lng': 146.157, 'surface_temperature': 71}
{'confidence': 100, 'lat': -37.634, 'lng': 149.237, 'surface_temperature': 71}
{'confidence': 98, 'lat': -37.605, 'lng': 149.302, 'surface_temperature': 83}
{'confidence': 99, 'lat': -37.6, 'lng': 149.325, 'surface_temperature': 86}
{'confidence': 95, 'lat': -37.618, 'lng': 149.281, 'surface_temperature': 76}
{'confidence': 100, 'lat': -37.606, 'lng': 149.312, 'surface_tempe

### 2.2c Finding climate and hotspot data on 15th and 16th of December 2022

In [119]:
# Find date, surface temperature (°C), air temperature (°C), relative humidity and max wind speed on 15th and 16th of December 2022.
for date in db.dates.find(
    {"$or": [{"_id": datetime(2022, 12, 15)}, {"_id": datetime(2022, 12, 16)}]}
):
    for hotspot_ref in date.get("hotspots"):
        for hotspot in db.hotspots.find({"_id": hotspot_ref.get("hotspot_id")}):
            print("-------------------------")
            print("Date: " + str(date.get("_id").date()))
            print("Surface Temperature: " + str(hotspot.get("surface_temperature")))
            print("Air Temperature: " + str(date.get("climate").get("air_temperature")))
            print("Relative Humidity: " + str(date.get("climate").get("relative_humidity")))
            print("Max Wind Speed: " + str(date.get("climate").get("max_wind_speed")))
            print()

-------------------------
Date: 2022-12-15
Surface Temperature: 42
Air Temperature: 18
Relative Humidity: 52.0
Max Wind Speed: 14.0

-------------------------
Date: 2022-12-15
Surface Temperature: 36
Air Temperature: 18
Relative Humidity: 52.0
Max Wind Speed: 14.0

-------------------------
Date: 2022-12-15
Surface Temperature: 38
Air Temperature: 18
Relative Humidity: 52.0
Max Wind Speed: 14.0

-------------------------
Date: 2022-12-15
Surface Temperature: 40
Air Temperature: 18
Relative Humidity: 52.0
Max Wind Speed: 14.0

-------------------------
Date: 2022-12-16
Surface Temperature: 43
Air Temperature: 18
Relative Humidity: 53.7
Max Wind Speed: 13.0

-------------------------
Date: 2022-12-16
Surface Temperature: 33
Air Temperature: 18
Relative Humidity: 53.7
Max Wind Speed: 13.0

-------------------------
Date: 2022-12-16
Surface Temperature: 54
Air Temperature: 18
Relative Humidity: 53.7
Max Wind Speed: 13.0

-------------------------
Date: 2022-12-16
Surface Temperature: 73
Ai

### 2.2d Finding climate and hotspot data with high confidence level

In [120]:
# Find datetime, air temperature (°C), surface temperature (°C) and confidence when the confidence is between 80 and 100.
for hotspot in db.hotspots.find({"confidence": {"$gt": 80, "$lt": 100}}):
    date_id = datetime.combine(hotspot.get("datetime").date(), datetime.min.time()) # Get date from datetime, date+min_time -> datetime
    for date in db.dates.find({"_id": date_id}):                                    # coz dates._id only have date and min_time
        print("------------------------------")
        print("Datetime: " + str(hotspot.get("datetime")))
        print("Surface Temperature: " + str(hotspot.get("surface_temperature")))
        print("Air Temperature: " + str(date.get("climate").get("air_temperature")))
        print("Confidence: " + str(hotspot.get("confidence")))
        print()

------------------------------
Datetime: 2022-12-27 00:02:15
Surface Temperature: 63
Air Temperature: 28
Confidence: 82

------------------------------
Datetime: 2022-12-27 00:02:14
Surface Temperature: 67
Air Temperature: 28
Confidence: 86

------------------------------
Datetime: 2022-12-16 15:38:39
Surface Temperature: 43
Air Temperature: 18
Confidence: 94

------------------------------
Datetime: 2022-12-16 04:35:13
Surface Temperature: 73
Air Temperature: 18
Confidence: 93

------------------------------
Datetime: 2022-12-16 04:34:58
Surface Temperature: 55
Air Temperature: 18
Confidence: 84

------------------------------
Datetime: 2022-12-16 04:34:58
Surface Temperature: 75
Air Temperature: 18
Confidence: 95

------------------------------
Datetime: 2022-12-16 04:34:57
Surface Temperature: 66
Air Temperature: 18
Confidence: 90

------------------------------
Datetime: 2022-12-16 00:21:12
Surface Temperature: 73
Air Temperature: 18
Confidence: 93

------------------------------
D

### 2.2e Finding records of top 10 highest surface temperature

In [121]:
for hotspot in db.hotspots.find().sort("surface_temperature", -1).limit(10):
    pprint(hotspot)

{'_id': ObjectId('64749dc234e617fd9c70d0f5'),
 'confidence': 100,
 'date': datetime.datetime(2022, 4, 18, 0, 0),
 'datetime': datetime.datetime(2022, 4, 18, 4, 52),
 'lat': -38.1665,
 'lng': 143.062,
 'surface_temperature': 124}
{'_id': ObjectId('64749dc234e617fd9c70d59f'),
 'confidence': 100,
 'date': datetime.datetime(2022, 4, 4, 0, 0),
 'datetime': datetime.datetime(2022, 4, 4, 4, 32, 50),
 'lat': -36.343,
 'lng': 142.1986,
 'surface_temperature': 123}
{'_id': ObjectId('64749dc234e617fd9c70d067'),
 'confidence': 100,
 'date': datetime.datetime(2022, 5, 1, 0, 0),
 'datetime': datetime.datetime(2022, 5, 1, 4, 14, 20),
 'lat': -36.9318,
 'lng': 143.0907,
 'surface_temperature': 122}
{'_id': ObjectId('64749dc234e617fd9c70d68e'),
 'confidence': 100,
 'date': datetime.datetime(2022, 3, 18, 0, 0),
 'datetime': datetime.datetime(2022, 3, 18, 3, 50, 50),
 'lat': -37.017,
 'lng': 148.1297,
 'surface_temperature': 121}
{'_id': ObjectId('64749dc234e617fd9c70d18e'),
 'confidence': 100,
 'date': 

### 2.2f Finding the number of fire each day

In [122]:
for date in db.dates.find():
    print("Date: "+ str(date.get("_id").date()))
    print("Number of fire: " + str(len(date.get("hotspots"))))
    print()

Date: 2022-03-06
Number of fire: 2

Date: 2022-03-07
Number of fire: 1

Date: 2022-03-08
Number of fire: 2

Date: 2022-03-09
Number of fire: 3

Date: 2022-03-10
Number of fire: 8

Date: 2022-03-12
Number of fire: 5

Date: 2022-03-13
Number of fire: 2

Date: 2022-03-14
Number of fire: 10

Date: 2022-03-15
Number of fire: 7

Date: 2022-03-17
Number of fire: 6

Date: 2022-03-18
Number of fire: 3

Date: 2022-03-19
Number of fire: 21

Date: 2022-03-24
Number of fire: 2

Date: 2022-03-25
Number of fire: 13

Date: 2022-03-26
Number of fire: 17

Date: 2022-03-28
Number of fire: 54

Date: 2022-03-29
Number of fire: 1

Date: 2022-03-31
Number of fire: 22

Date: 2022-04-01
Number of fire: 7

Date: 2022-04-02
Number of fire: 5

Date: 2022-04-03
Number of fire: 72

Date: 2022-04-04
Number of fire: 89

Date: 2022-04-05
Number of fire: 49

Date: 2022-04-06
Number of fire: 118

Date: 2022-04-07
Number of fire: 39

Date: 2022-04-08
Number of fire: 20

Date: 2022-04-11
Number of fire: 24

Date: 2022-04-

### 2.2g Finding records of fire where confidence is less than 70

In [123]:
for hotspot in db.hotspots.find({"confidence": {"$lt": 70}}):
    pprint(hotspot)

{'_id': ObjectId('64749dc234e617fd9c70cc54'),
 'confidence': 67,
 'date': datetime.datetime(2022, 12, 27, 0, 0),
 'datetime': datetime.datetime(2022, 12, 27, 0, 2, 15),
 'lat': -35.554,
 'lng': 143.307,
 'surface_temperature': 53}
{'_id': ObjectId('64749dc234e617fd9c70cc57'),
 'confidence': 65,
 'date': datetime.datetime(2022, 12, 24, 0, 0),
 'datetime': datetime.datetime(2022, 12, 24, 13, 12, 1),
 'lat': -35.646,
 'lng': 142.282,
 'surface_temperature': 32}
{'_id': ObjectId('64749dc234e617fd9c70cc5a'),
 'confidence': 69,
 'date': datetime.datetime(2022, 12, 16, 0, 0),
 'datetime': datetime.datetime(2022, 12, 16, 15, 38, 39),
 'lat': -37.624,
 'lng': 149.332,
 'surface_temperature': 33}
{'_id': ObjectId('64749dc234e617fd9c70cc5b'),
 'confidence': 62,
 'date': datetime.datetime(2022, 12, 16, 0, 0),
 'datetime': datetime.datetime(2022, 12, 16, 4, 35, 15),
 'lat': -37.82,
 'lng': 142.323,
 'surface_temperature': 54}
{'_id': ObjectId('64749dc234e617fd9c70cc5f'),
 'confidence': 59,
 'date':

### 2.2h Finding the average surface temperature of each day

In [134]:
from bson.son import SON
for result in db.hotspots.aggregate([{
    "$group": {
        "_id": "$date",
        "average temperature": {"$avg": "$surface_temperature"}
        }
    },
    {"$sort": SON([("_id", 1)])}
]):
    pprint(result)

{'_id': datetime.datetime(2022, 3, 6, 0, 0), 'average temperature': 60.5}
{'_id': datetime.datetime(2022, 3, 7, 0, 0), 'average temperature': 64.0}
{'_id': datetime.datetime(2022, 3, 8, 0, 0), 'average temperature': 51.5}
{'_id': datetime.datetime(2022, 3, 9, 0, 0),
 'average temperature': 46.666666666666664}
{'_id': datetime.datetime(2022, 3, 10, 0, 0), 'average temperature': 69.375}
{'_id': datetime.datetime(2022, 3, 12, 0, 0), 'average temperature': 88.2}
{'_id': datetime.datetime(2022, 3, 13, 0, 0), 'average temperature': 38.5}
{'_id': datetime.datetime(2022, 3, 14, 0, 0), 'average temperature': 65.6}
{'_id': datetime.datetime(2022, 3, 15, 0, 0), 'average temperature': 46.0}
{'_id': datetime.datetime(2022, 3, 17, 0, 0), 'average temperature': 59.5}
{'_id': datetime.datetime(2022, 3, 18, 0, 0),
 'average temperature': 79.33333333333333}
{'_id': datetime.datetime(2022, 3, 19, 0, 0),
 'average temperature': 65.57142857142857}
{'_id': datetime.datetime(2022, 3, 24, 0, 0), 'average temp

### 2.2i Finding records with the 10 lowest GHI

In [138]:
for date in db.dates.find().sort("climate.ghi", 1).limit(10):
    pprint(date)

{'_id': datetime.datetime(2022, 8, 2, 0, 0),
 'climate': {'air_temperature': 5,
             'ghi': 47,
             'max_wind_speed': 5.1,
             'precipitation': ' 0.00I',
             'relative_humidity': 38.6,
             'windspeed_knots': 1.8},
 'hotspots': [{'hotspot_id': ObjectId('64749dc234e617fd9c70cd74')},
              {'hotspot_id': ObjectId('64749dc234e617fd9c70cd75')}]}
{'_id': datetime.datetime(2022, 6, 30, 0, 0),
 'climate': {'air_temperature': 5,
             'ghi': 48,
             'max_wind_speed': 11.1,
             'precipitation': ' 0.00A',
             'relative_humidity': 34.9,
             'windspeed_knots': 4.3},
 'hotspots': [{'hotspot_id': ObjectId('64749dc234e617fd9c70cd8f')},
              {'hotspot_id': ObjectId('64749dc234e617fd9c70cd90')},
              {'hotspot_id': ObjectId('64749dc234e617fd9c70cd91')},
              {'hotspot_id': ObjectId('64749dc234e617fd9c70cd92')},
              {'hotspot_id': ObjectId('64749dc234e617fd9c70cd93')},
     

### 2.2j